In [1]:
# ปรับแต่งค่าการทำงานของ Spark
from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("KafkaSubscribe").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "1000m").\
        config("spark.executor.cores", "2").\
        config("spark.cores.max", "6").\
        config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0").\
        config("spark.jars.packages", "com.microsoft.azure:spark-mssql-connector:1.0.2").\
        getOrCreate()



Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
com.microsoft.azure#spark-mssql-connector added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2cdd9d60-dd93-4271-b39c-854cf0f79f49;1.0
	confs: [default]
	found com.microsoft.azure#spark-mssql-connector;1.0.2 in central
	found com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8 in central
downloading https://repo1.maven.org/maven2/com/microsoft/azure/spark-mssql-connector/1.0.2/spark-mssql-connector-1.0.2.jar ...
	[SUCCESSFUL ] com.microsoft.azure#spark-mssql-connector;1.0.2!spark-mssql-connector.jar (48ms)
downloading https://repo1.maven.org/maven2/com/microsoft/sqlserver/mssql-jdbc/8.4.1.jre8/mssql-jdbc-8.4.1.jre8.jar ...
	[SUCCESSFUL ] com.microsoft.sqlserver#mssql-jdbc;8.4.1.jre8!mssql-jdbc.jar (143ms)
:: reso

In [4]:
server_name = "jdbc:sqlserver://10.128.0.66"
database_name = "testDB"
url = server_name + ";" + "databaseName=" + database_name + ";"

table_name = "spark_loan_stat_10000"
username = "SA"
password = "Passw0rd123456" # Please specify password here

read_result_df = spark.read.format("jdbc")        .option("url", url)         .option("dbtable", table_name)         .option("user", username)         .option("password", password)        .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver").load()


In [5]:
read_result_df.count()

735740

In [6]:
read_result_df.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
loan_amnt,735740,14503.327024492348,9240.777415179264,1000,40000
term,735740,None,None,36 months,60 months
int_rate,735740,13.208798923640442,5.157202466162041,5.31,30.99
installment,735740,446.5535269525436,282.87812370846495,30.12,1715.42
grade,735740,None,None,A,G
emp_length,735740,None,None,1 year,n/a
home_ownership,735740,None,None,ANY,RENT
annual_inc,735740,79534.12686194837,77998.95354640124,0.0,9573072.0
verification_status,735740,None,None,Not Verified,Verified


In [7]:
read_result_df.printSchema()

root
 |-- loan_amnt: integer (nullable = true)
 |-- term: string (nullable = true)
 |-- int_rate: double (nullable = true)
 |-- installment: double (nullable = true)
 |-- grade: string (nullable = true)
 |-- emp_length: string (nullable = true)
 |-- home_ownership: string (nullable = true)
 |-- annual_inc: double (nullable = true)
 |-- verification_status: string (nullable = true)
 |-- loan_status: string (nullable = true)
 |-- purpose: string (nullable = true)
 |-- addr_state: string (nullable = true)
 |-- dti: string (nullable = true)
 |-- delinq_2yrs: string (nullable = true)
 |-- earliest_cr_line: string (nullable = true)
 |-- inq_last_6mths: double (nullable = true)
 |-- open_acc: integer (nullable = true)
 |-- pub_rec: integer (nullable = true)
 |-- revol_bal: integer (nullable = true)
 |-- revol_util: double (nullable = true)
 |-- total_acc: integer (nullable = true)
 |-- last_credit_pull_d: string (nullable = true)



In [9]:
# Set the connection properties
properties = {
    "user": username,
    "password": password,
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}


# Assuming 'read_result_df' is your DataFrame
read_result_df.write.jdbc(url=url, table='loan_stat_jun2023', mode='overwrite', properties=properties)